In [ ]:
# !pip install pycoingecko binance mplfinance

In [ ]:
!pip install cryptocompare plotly

  Preparing metadata (setup.py) ... done
  Created wheel for cryptocompare: filename=cryptocompare-0.7.6-py3-none-any.whl size=6289 sha256=5c9327cad743e27abfacee33b4f6363e367747c8060dcc3c0b3ee7e8c2281ba7
  Stored in directory: /root/.cache/pip/wheels/97/81/6a/261bc499b9d9bf8b391eea52c90a4746b477539564b0a66c94
Successfully built cryptocompare


In [ ]:
# from pycoingecko import CoinGeckoAPI
import cryptocompare
import pandas as pd
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
from google.colab import  drive
import os
import numpy as np
from itertools import islice
import matplotlib.dates as mdates
import plotly.graph_objects as go
from plotly.subplots import make_subplots

drive.mount('/drive')
folder_path = '/drive/My Drive/Colab Notebooks/Crypto Project/Data/cryptocompare'
plot_results = False
CoinGecko = False

Mounted at /drive


In [ ]:
cryptocurrency_release_dates = {
    "Bitcoin (BTC)": "2012-01-01",
    "Ethereum (ETH)": "2015-07-30",
    "Tether (USDT)": "2014-10-06",
    "Binance Coin (BNB)": "2017-07-25",
    "USD Coin (USDC)": "2018-09-26",
    "Ripple (XRP)": "2012-01-01",
    "Solana (SOL)": "2020-03-01",
    "Cardano (ADA)": "2017-09-29",
    "Dogecoin (DOGE)": "2013-12-06",
    "Toncoin (TON)": "2021-08-01",
    "Polkadot (DOT)": "2020-05-26",
    "Litecoin (LTC)": "2012-01-01",
    "Chainlink (LINK)": "2017-09-19",
    "Uniswap (UNI)": "2020-09-16",
    "Bitcoin Cash (BCH)": "2017-08-01",
    "Stellar (XLM)": "2014-07-31",
    "VeChain (VET)": "2018-08-01",
    "TRON (TRX)": "2017-09-13",
    "Filecoin (FIL)": "2020-10-15",
    "Aave (AAVE)": "2020-10-01"
}

In [ ]:
for coin in cryptocurrency_release_dates.keys():
  coin_short = coin.split('(')[1][:-1]
  print(f"{coin_short.lower()}")

btc
eth
usdt
bnb
usdc
xrp
sol
ada
doge
ton
dot
ltc
link
uni
bch
xlm
vet
trx
fil
aave


#Darstellung mit CoinGecko (Depricated)

In [ ]:
if CoinGecko:
  cg = CoinGeckoAPI()
  coin_list = cg.get_coins_list()

  coins_df = pd.DataFrame(coin_list)
  display(coins_df)

## Initial Data Loading

In [ ]:
if CoinGecko:
  # Initialisiere die CoinGecko API
  cg = CoinGeckoAPI()

  coin_marked = cg.get_coins_markets(vs_currency='usd')
  coins_df = pd.DataFrame(coin_marked)
  top_10 = coins_df.sort_values(by='market_cap', ascending=False).head(10)['id'].values.tolist()
  print(top_10)

  """
  crypto_currencies = ['bitcoin', 'ethereum','litecoin', 'solana']
  crypto_data = {}
  file_path = '/drive/My Drive/Colab Notebooks/Crypto Project/Data'
  for currency in top_10:
    data = cg.get_coin_market_chart_range_by_id(id=currency,
                                                vs_currency='eur',
                                                from_timestamp="1728793948", # '1262304000'# year = 2010
                                                to_timestamp='1729312348')
                                                # interval='daily')

    prices = data['prices']  # Enthält Zeitstempel und Preise
    volume = data['total_volumes']  # Enthält Zeitstempel und Handelsvolumen

    df = pd.DataFrame(prices, columns=['Timestamp', 'Price'])
    df['Volume'] = [v[1] for v in volume]

    # Zeitstempel in Datumsformat umwandeln
    df['Date'] = pd.to_datetime(df['Timestamp'], unit='ms')
    df.set_index('Date', inplace=True)
    df = df[['Price', 'Volume']]
    df.to_csv(f'{file_path}/{currency}_file_eur.csv')
  """
  for coin, release_date in cryptocurrency_release_dates.items():
    coin_name = coin.split('(')[0]
    coin_symbol = coin.split('(')[1][:-1]
    # date_range = pd.date_range(start=release_date, end="2024-10-01", freq='1ME')
    date_range = pd.date_range(start=datetime.strptime(release_date, '%Y-%m-%d')-timedelta(days=1), end="2024-10-01", freq='1YE')
    print(release_date, date_range[0], date_range[-1])

## Update Data with Coin Gecko (DEPRICATED)

In [ ]:
if CoinGecko:
  # Initialisiere die CoinGecko API
  cg = CoinGeckoAPI()


  coin_marked = cg.get_coins_markets(vs_currency='usd')
  coins_df = pd.DataFrame(coin_marked)
  top_10 = coins_df.sort_values(by='market_cap', ascending=False).head(10)['id'].values.tolist()

  crypto_currencies = ['bitcoin', 'ethereum','litecoin', 'solana']
  crypto_data = {}
  file_path = '/drive/My Drive/Colab Notebooks/Crypto Project/Data'
  for currency in top_10:
    # historical_data = cg.get_coin_market_chart_range_by_id(id='bitcoin',vs_currency='usd',from_timestamp='1605096000',to_timestamp='1605099600')
    data = cg.get_coin_market_chart_by_id(id=currency,
                                          vs_currency='usd',
                                          days=2,
                                          interval='daily',
                                          include_market_cap='true')
    prices = data['prices']  # Enthält Zeitstempel und Preise
    volume = data['total_volumes']  # Enthält Zeitstempel und Handelsvolumen

    df = pd.DataFrame(prices, columns=['Timestamp', 'Price'])
    df['Volume'] = [v[1] for v in volume]

    # Zeitstempel in Datumsformat umwandeln
    df['Date'] = pd.to_datetime(df['Timestamp'], unit='ms')
    df.set_index('Date', inplace=True)
    df = df[['Price', 'Volume']]
    if os.path.exists(file_path):
      df_old = pd.read_csv(f'{folder_path}/{currency_file}', parse_dates=True, index_col='Date')
      df_old.index = pd.to_datetime(df_old.index)
      last_entry_date = df_old.index[-1]
      new_entry_date = df.index[0]
      print(last_entry_date, new_entry_date)
      if new_entry_date - last_entry_date <= timedelta(days=1):
        filtered_new_df = df[~df.index.isin(df_old.index)]
        if not filtered_new_df.empty:
          df = pd.concat([df_old, df], ignore_index=True)
        else:
          print('no new data')
      else:
        print('too much time gap')
    df.to_csv(f'{file_path}/{currency}_file.csv')
    crypto_data[currency] = df


# Get hostorical data with CryptoCompare

---



In [ ]:
def get_historical_data(crypto_coin, currency, start_date, end_date):
  API_KEY = 'a6de8441a437af764b5763b81477a3034fb71c70b6110f41f5bc4496bdf21f37'  # Replace with your CryptoCompare API key
  url = 'https://min-api.cryptocompare.com/data/v2/histoday'
  if type(start_date) == str:
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.strptime(end_date, '%Y-%m-%d')

  limit = end_date - start_date
  limit = limit.days
  if limit > 2000:
    print('limit is too high')
    return
  cryptocompare.cryptocompare._set_api_key_parameter(API_KEY)
  try:
    cc = cryptocompare.get_historical_price_day(crypto_coin, currency,
                                                limit=limit, exchange='CCCAGG',
                                                toTs=end_date)
    df_cc = pd.DataFrame(cc)
    df_cc['time'] = pd.to_datetime(df_cc['time'], unit='s')
  except KeyError:
    print('ERROR')
    return
  return df_cc


for coin, release_date in cryptocurrency_release_dates.items():
  coin_name = coin.split('(')[0]
  coin_symbol = coin.split('(')[1][:-1]
  date_range = pd.date_range(start=release_date, end="2024-10-01", freq='1YE')
  # date_range = pd.date_range(start=datetime.strptime(release_date, '%Y-%m-%d')-timedelta(days=1), end="2024-10-01", freq='1YE')
  print(f'Coin: {coin}')
  for sd, ed in zip(date_range[:-1], date_range[1:]):
    # print(f'Coin: {coin} \n Start: {sd}\n End: {ed}')
    hist_data = get_historical_data(coin_symbol, 'USD', sd, ed)
    if hist_data is None:
      continue
    # print(hist_data)
    # hist_data.set_index('time', inplace=True)
    hist_data = hist_data[['time', 'high', 'low', 'open', 'volumefrom', 'volumeto', 'close']].iloc[:-1]


    if os.path.exists(folder_path):
      if os.path.exists(f'{folder_path}/{coin_symbol}_file.csv'):
        df_old = pd.read_csv(f'{folder_path}/{coin_symbol}_file.csv', parse_dates=['time'], index_col=0, date_format='%Y-%m-%d')
        last_entry_date = df_old.index[-1]
        new_entry_date = hist_data.index[0]
        # print(last_entry_date, new_entry_date)
        # print(f" New Data:\n{hist_data.head()}\n\n Old Data:\n{df_old.head()}\n\n\n")
        hist_data = pd.concat([df_old, hist_data], axis=0)
        hist_data.to_csv(f'{folder_path}/{coin_symbol}_file.csv')
      else:
        hist_data.to_csv(f'{folder_path}/{coin_symbol}_file.csv')

Coin: Bitcoin (BTC)
Coin: Ethereum (ETH)
Coin: Tether (USDT)
Coin: Binance Coin (BNB)
Coin: USD Coin (USDC)
Coin: Ripple (XRP)
Coin: Solana (SOL)
Coin: Cardano (ADA)
Coin: Dogecoin (DOGE)
Coin: Toncoin (TON)
Coin: Polkadot (DOT)
Coin: Litecoin (LTC)
Coin: Chainlink (LINK)
Coin: Uniswap (UNI)
Coin: Bitcoin Cash (BCH)
Coin: Stellar (XLM)
Coin: VeChain (VET)
Coin: TRON (TRX)
Coin: Filecoin (FIL)
Coin: Aave (AAVE)


## Connection with traditional API Request

In [ ]:
  """
  # wiith traditional API request
  params = {
      'fsym': crypto_coin,
      'tsym': currency,
      'limit': limit,         # Number of days of data (limit is up to 2000 for free tier)
      'toTs': int(end_date.timestamp()),  # The Unix timestamp for the last date
      'api_key': API_KEY   # Your API key
  }

  # Make the API request
  response = requests.get(url, params=params)

  # Check if the request was successful
  if response.status_code == 200:
      if response.json()['Response'] == 'Error':
        print(response.json()['Message'])
        return None
      data = response.json()['Data']['Data']
      df = pd.DataFrame(data)
      df['time'] = pd.to_datetime(df['time'], unit='s')
      print(df)
      return df

  else:
      print(f"Failed to retrieve data: {response.status_code}")
      return None
  """

'\n# wiith traditional API request\nparams = {\n    \'fsym\': crypto_coin,\n    \'tsym\': currency,\n    \'limit\': limit,         # Number of days of data (limit is up to 2000 for free tier)\n    \'toTs\': int(end_date.timestamp()),  # The Unix timestamp for the last date\n    \'api_key\': API_KEY   # Your API key\n}\n\n# Make the API request\nresponse = requests.get(url, params=params)\n\n# Check if the request was successful\nif response.status_code == 200:\n    if response.json()[\'Response\'] == \'Error\':\n      print(response.json()[\'Message\'])\n      return None\n    data = response.json()[\'Data\'][\'Data\']\n    df = pd.DataFrame(data)\n    df[\'time\'] = pd.to_datetime(df[\'time\'], unit=\'s\')\n    print(df)\n    return df\n\nelse:\n    print(f"Failed to retrieve data: {response.status_code}")\n    return None\n'

## Get latest Updates in Crypto Currencies

In [ ]:
for coin, release_date in cryptocurrency_release_dates.items():
  coin_name = coin.split('(')[0]
  coin_symbol = coin.split('(')[1][:-1]
  release_date = pd.to_datetime(release_date)
  try:
    df_old = pd.read_csv(f'{folder_path}/{coin_symbol}_file.csv', parse_dates=['time'], index_col=0, date_format='%Y-%m-%d')
    df_old = df_old[['time', 'high', 'low', 'open', 'volumefrom', 'volumeto', 'close']]

  except FileNotFoundError:
    print(f"No file found for {coin_symbol}")
    continue

  if not df_old.empty:
      last_date = df_old['time'].max()
  else:
      last_date = release_date

  # Ensure last_date is at least one day before today
  today = pd.Timestamp.today().normalize()
  last_date = pd.to_datetime(last_date)
  if last_date >= today:
      print(f"No update needed for {coin_symbol}")
  else:
      # Set the end_date for fetching new data (one day before today)
      days_to_fetch = (today - last_date).days + 2 # just to be save
      end_date = datetime.now() - timedelta(days=1)
      last_timestamp = int(last_date.timestamp())

      new_data = cryptocompare.get_historical_price_day(coin_symbol, 'USD',
                                                        limit=days_to_fetch, exchange='CCCAGG',
                                                        toTs=int(end_date.timestamp()))
      df_new = pd.DataFrame(new_data)
      df_new['time'] = pd.to_datetime(df_new['time'], unit='s')
      df_new = df_new[['time', 'high', 'low', 'open', 'volumefrom', 'volumeto', 'close']]

      # Filter new data to ensure it's only after the last date in df_old
      df_new = df_new[df_new['time'] > last_date]

      # Append new data to old data
      df_updated = pd.concat([df_old, df_new], axis=0)

      df_updated.to_csv(f'{folder_path}/{coin_symbol}_file.csv')

      print(f"Data updated for {coin_symbol}")


Data updated for BTC
Data updated for ETH
Data updated for USDT
Data updated for BNB
Data updated for USDC
Data updated for XRP
Data updated for SOL
Data updated for ADA
Data updated for DOGE
Data updated for TON
Data updated for DOT
Data updated for LTC
Data updated for LINK
Data updated for UNI
Data updated for BCH
Data updated for XLM
Data updated for VET
Data updated for TRX
Data updated for FIL
Data updated for AAVE


#Build Dashboard

## Load Data

In [ ]:

coin_dict = {}
for coin, release_date in cryptocurrency_release_dates.items():
  coin_name = coin.split('(')[0]
  coin_symbol = coin.split('(')[1][:-1]
  release_date = pd.to_datetime(release_date)
  try:
    currency_df = pd.read_csv(f'{folder_path}/{coin_symbol}_file.csv', parse_dates=['time'], index_col=0, date_format='%Y-%m-%d')
    currency_df = currency_df[['time', 'high', 'low', 'open', 'volumefrom', 'volumeto', 'close']]
  except FileNotFoundError:
    print(f"No file found for {coin_symbol}")
    continue

  coin_dict[coin_name] = currency_df

## Classic Matplotlib All-In-One

In [ ]:
if plot_results:
  fig, ax1 = plt.subplots(figsize=(12, 8))
  colors = plt.cm.get_cmap('tab20').colors
  for nr, (coin, prices) in enumerate(coin_dict.items()):
    if nr > 0:
      ax_tmp = ax1.twinx()
      # ax_tmp.step(prices.index, prices['Price'], where='mid', label=coin, linewidth=2, color=colors[nr])
      ax_tmp.plot(prices['time'], prices['close'], label=coin, linewidth=2, color=colors[nr])
      ax_tmp.set_ylabel(coin, color=colors[nr])
      ax_tmp.tick_params(axis='y', labelcolor=colors[nr])
    else:
      # ax1.step(prices.index, prices['Price'], where='mid', label=coin, linewidth=2, color=colors[nr])
      ax1.plot(prices['time'], prices['close'], label=coin, linewidth=2, color=colors[nr])
      ax1.set_xlabel('Date')
      ax1.set_ylabel(coin, color=colors[nr])
      ax1.tick_params(axis='y', labelcolor=colors[nr])

  # Formatting
  fig.tight_layout()
  plt.title('Bitcoin, Ethereum, and Solana Prices with Step Chart')
  plt.xticks(rotation=45)
  plt.legend()

  # Show the plot
  plt.show()

## Interactive Plot with Plotly

In [ ]:

def plotly_plot(coin_dict):

  fig = go.Figure()

  # Loop through each coin in your dictionary
  colors = ['blue', 'orange', 'green', 'red', 'purple']  # Define some colors for different coins
  for idx, (coin, prices) in enumerate(coin_dict.items()):
      fig.add_trace(go.Scatter(
          x=prices['time'],
          y=prices['close'],
          mode='lines',
          name=coin,
          line=dict(width=2, color=colors[idx % len(colors)]),
          visible=True  # Initially make all traces visible
      ))

  # Add checkboxes (interactive visibility control)
  fig.update_layout(
      updatemenus=[
          dict(
              type="buttons",
              direction="down",
              showactive=True,
              buttons=list([
                  dict(label="Show All",
                      method="update",
                      args=[{"visible": [True] * len(coin_dict)},  # Show all coins
                            {"title": "All Coins"}]),
                  dict(label="Hide All",
                      method="update",
                      args=[{"visible": [False] * len(coin_dict)},  # Hide all coins
                            {"title": "No Coins"}]),
                  # Add individual buttons for each coin
                  *[dict(label=f"{coin}",
                        method="update",
                        args=[{"visible": [i == idx for i in range(len(coin_dict))]},  # Show only the selected coin
                              {"title": f"{coin} Price"}])
                    for idx, coin in enumerate(coin_dict)]
              ]),
          )
      ]
  )

  # Layout formatting
  fig.update_layout(
      title="Cryptocurrency Prices",
      xaxis_title="Date",
      yaxis_title="Price (USD)",
      legend_title="Cryptocurrencies",
      xaxis=dict(tickformat='%Y-%m-%d'),  # Format x-axis as dates
      hovermode="x unified",
  )
  return fig


In [ ]:
if plot_results:
  fig = plotly_plot(coin_dict)
  fig.show()

In [ ]:
# plt.close('all')

In [ ]:
if plot_results:
  stats = {}
  for coin, prices in coin_dict.items():
    stats[coin] = {'min': min(prices['open']), 'max': max(prices['open'])}
    print(coin)
    print(stats[coin])
    print()

# Grouping

In [ ]:
if plot_results:
  df = pd.DataFrame(stats).T
  df['range'] = df['max'] - df['min']

  # Define thresholds for grouping (these can be adjusted)
  low_threshold = df['range'].quantile(0.33)  # Lower 33% for low volatility
  high_threshold = df['range'].quantile(0.66)  # Upper 33% for high volatility

  # Classify the coins into three groups
  def classify_volatility(row):
      if row['range'] <= low_threshold:
          return 'Low Volatility'
      elif row['range'] <= high_threshold:
          return 'Medium Volatility'
      else:
          return 'High Volatility'

  df['volatility_group'] = df.apply(classify_volatility, axis=1)

  # Display the coins grouped by volatility
  grouped_coins = df.groupby('volatility_group').apply(lambda x: list(x.index))
  for group, coins in grouped_coins.items():
      print(f"{group}: {', '.join(coins)}")


In [ ]:
if plot_results:
  df = pd.DataFrame(stats).T
  df['range'] = df['max'] - df['min']

  # Sort coins by their range values
  df_sorted = df.sort_values('range')

  # Grouping coins into groups of 2-3 similar ranges
  grouped_coins = []
  group_size = 3  # Target group size

  # Loop through sorted coins and create groups
  for i in range(0, len(df_sorted), group_size):
      group = df_sorted.iloc[i:i + group_size]
      grouped_coins.append(list(group.index))

  # Print the groups
  for idx, group in enumerate(grouped_coins):
      print(f"Group {idx + 1}: {', '.join(group)}")

## Similar Range Plot

## Clear all GO-**Figures**

In [ ]:
def plot_group(group_coins):
  fig = go.Figure()

  # Loop through each coin in your dictionary
  colors = ['blue', 'orange', 'green', 'red', 'purple']  # Define some colors for different coins

  for idx, (coin, prices) in enumerate(group_coins.items()):
    go.Scatter(
        x=prices['time'],
        y=prices['close'],
        mode='lines',
        name=coin,
        line=dict(width=2, color=colors[idx]))



  # Layout formatting
  fig.update_layout(
      title="Cryptocurrency Prices",
      xaxis_title="Date",
      yaxis_title="Price (USD)",
      legend_title="Cryptocurrencies",
      xaxis=dict(tickformat='%Y-%m-%d'),  # Format x-axis as dates
      hovermode="x unified",
  )

# Show the figure


## Matplotlib Pyplot Figure

In [ ]:
def plot_group_pyplot(group_coins):
    fig, ax = plt.subplots(figsize=(10, 6))

    # Define some colors for the different coins
    colors = ['blue', 'orange', 'green', 'red', 'purple']

    # Loop through each coin in your dictionary
    for idx, (coin, prices) in enumerate(group_coins.items()):
        ax.plot(prices['time'], prices['close'], label=coin, color=colors[idx], linewidth=2)

    # Formatting the plot
    ax.set_title("Cryptocurrency Prices", fontsize=16)
    ax.set_xlabel("Date", fontsize=12)
    ax.set_ylabel("Price (USD)", fontsize=12)
    ax.legend(title="Cryptocurrencies")

    # Format the x-axis for dates
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    ax.xaxis.set_major_locator(mdates.AutoDateLocator())

    # Rotate the x-axis labels for better readability
    plt.xticks(rotation=45)

    # Display grid and plot
    ax.grid(True)
    plt.tight_layout()  # Adjust layout to prevent overlap
    plt.show()


In [ ]:
if plot_results:
  for idx, group in enumerate(grouped_coins):
    plot_group_pyplot({coin: prices for coin, prices in coin_dict.items() if coin in grouped_coins})
    plt.show()

# Normalize Data

In [ ]:
if plot_results:
  norm_coin_dict = {}
  for coin, prices in coin_dict.items():
    mean_close = prices['close'].mean()
    std_close = prices['close'].std()
    max_close = prices['close'].max()
    df_std_norm = pd.DataFrame()

    df_std_norm['zscore_close'] = (prices['close'] - mean_close) / std_close
    # Step 2: Apply a Non-Linear Scaling (e.g., hyperbolic tangent)
    # This will compress large deviations from the mean and smooth out extreme values
    df_std_norm['scaled_close'] = np.tanh(df_std_norm['zscore_close'])
    c = 1e-5  # You can adjust this constant based on your data
    df_std_norm['log_close'] = np.log(prices['close'] + c)

    # Step 3 (Optional): Rescale back to a more interpretable range if needed
    # For example, you can map the values back to 0-1 range if you prefer
    df_std_norm['close'] = (df_std_norm['log_close'] - df_std_norm['log_close'].min()) / (df_std_norm['log_close'].max() - df_std_norm['log_close'].min())

    df_std_norm['time'] = prices['time']

    df_norm = pd.DataFrame()
    df_norm['close'] = prices['close'] / max_close
    df_norm['time'] = prices['time']
    norm_coin_dict[f"std_norm_{coin}"] = df_std_norm
    norm_coin_dict[f"max_norm_{coin}"] = df_norm

In [ ]:
if plot_results:
  fig = plotly_plot(norm_coin_dict)
  fig.show()